In [1]:
import os
import torch
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
from torch.utils.data import DataLoader, random_split

# Step 1: Load Data
data_dir = 'C:\\Users\\singh\\Major_Project\\work_done\\stft_files\\New folder'  

# Step 2: Data Preprocessing
transform = transforms.Compose([
    transforms.Resize((227, 227)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  
])

# Load dataset
dataset = datasets.ImageFolder(root=data_dir, transform=transform)

# Step 3: Split Dataset into Training and Testing
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# Step 4: Define Model
model = models.alexnet(pretrained=True)
num_features = model.classifier[6].in_features
model.classifier[6] = nn.Linear(num_features, 3)  # 3 classes: AD, FTD, CN

# Step 5: Training
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)

    # Step 6: Evaluation
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print('Epoch [%d/%d], Loss: %.4f, Validation Accuracy: %.2f %%' %
          (epoch+1, num_epochs, running_loss/len(train_dataset), 100 * correct / total))


C:\Users\singh\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\singh\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch [1/10], Loss: 0.9635, Validation Accuracy: 62.58 %
Epoch [2/10], Loss: 0.7312, Validation Accuracy: 70.11 %
Epoch [3/10], Loss: 0.4056, Validation Accuracy: 82.70 %
Epoch [4/10], Loss: 0.1788, Validation Accuracy: 96.71 %
Epoch [5/10], Loss: 0.0949, Validation Accuracy: 99.24 %
Epoch [6/10], Loss: 0.0495, Validation Accuracy: 98.34 %
Epoch [7/10], Loss: 0.0374, Validation Accuracy: 98.90 %
Epoch [8/10], Loss: 0.0211, Validation Accuracy: 99.62 %
Epoch [9/10], Loss: 0.0188, Validation Accuracy: 98.72 %
Epoch [10/10], Loss: 0.0105, Validation Accuracy: 99.91 %
